## Set up google colab environment

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/My Drive/Hyperspectral_Image_Classification/V1/SGCNN_12')

In [4]:
from sample_extraction_V1_SGCNN_12 import *
import scipy.io as sio

## Load Indian Pines Dataset - Source

In [5]:
uIndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_corrected.mat')
gt_IndianPines = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Indian_pines_gt.mat')

In [6]:
data_source = uIndianPines['indian_pines_corrected']
ground_truth_source = gt_IndianPines['indian_pines_gt']

In [7]:
data_source.shape

(145, 145, 200)

In [8]:
ground_truth_source.shape

(145, 145)

# Load target dataset

In [9]:
uBotswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana.mat')
gt_Botswana = sio.loadmat('/content/drive/My Drive/Hyperspectral_Image_Classification/Datasets/Botswana_gt.mat')

In [10]:
data_target = uBotswana['Botswana']
ground_truth_target = gt_Botswana['Botswana_gt']

In [11]:
data_target.shape

(1476, 256, 145)

In [12]:
ground_truth_target.shape

(1476, 256)

## Distrubution of samples for each class in Source

In [13]:
class_distribution_source = pd.DataFrame(np.unique(ground_truth_source, return_counts = True))
class_distribution_source = class_distribution_source.transpose()
class_distribution_source.columns = ['class','samples']
class_distribution_source

,class,samples
0,0,10776
1,1,46
2,2,1428
3,3,830
4,4,237
5,5,483
6,6,730
7,7,28
8,8,478
9,9,20


In [14]:
classes_source , counts_source = np.unique(ground_truth_source, return_counts = True)
classes_source = classes_source[[2,3,5,6,8,10,11,12,14]] ## Dropping classes with small number of samples
classes_source

array([ 2,  3,  5,  6,  8, 10, 11, 12, 14], dtype=uint8)

# Class distribution of samples in Target

In [15]:
class_distribution_target = pd.DataFrame(np.unique(ground_truth_target, return_counts = True))
class_distribution_target = class_distribution_target.transpose()
class_distribution_target.columns = ['class','samples']
class_distribution_target

,class,samples
0,0,374608
1,1,270
2,2,101
3,3,251
4,4,215
5,5,269
6,6,269
7,7,259
8,8,203
9,9,314


In [16]:
classes_target , counts_target = np.unique(ground_truth_target, return_counts = True)
classes_target = classes_target[1:] ## Dropping classes with small number of samples
classes_target

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14],
      dtype=uint8)

## Source : Indian Pines

## Train model for samples extracted with different overlap ratios and a percent of  samples picked from each class to be present in the training set. 

## Model except the final fully connected layer is saved for transfer learning.

In [17]:
pretrain_source_models(training_set_size = [200],
                      classes = classes_source,
                      cube_size = 20,
                      overlap_ratio = 1,
                      data = data_source,
                      ground_truth = ground_truth_source,
                      batch_size = 20,
                      channels = 64,
                      epochs = 50,
                      Verbosity = 1,
                      accuracies = [],
                      learning_rate = 0.0001,
                      source_dataset = 'indian_pines')


Model training starts for data with 200 samples from each class in training set

Samples per class: [1368, 523, 323, 730, 356, 837, 2224, 460, 1085]
Total number of samples 7906.

Total number of samples in training set 1800.
unique classes in training set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in training set: [200 200 200 200 200 200 200 200 200]

Total number of samples in test set 6106.
unique classes in test set: [ 2  3  5  6  8 10 11 12 14]
Samples per class in test set: [1168  323  123  530  156  637 2024  260  885]

X_train => (1800, 20, 20, 64)
X_test  => (6106, 20, 20, 64)
y_train => (1800, 9)
y_test  => (6106, 9)

Group convolution output:  (None, 20, 20, 64)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            

# Fine tune on botswana

In [18]:
transfer_results, confusion_matrixes = transfer_learning(source_dataset = 'indian_pines',
                                        target_dataset = 'botswana',
                                        data = data_target,
                                        ground_truth = ground_truth_target,
                                        training_samples_from_each_class = [15,30,45,60,75],
                                        source_training_size = [200],
                                        classes = classes_target,
                                        overlap_ratio = 1,
                                        channels = 64,
                                        cube_size = 20,
                                        learning_rate = 0.0001,
                                        epochs = 150,
                                        batch_size = 20,
                                        test_accuracies = [],
                                        )


Model training starts for data with 15 samples from each class in training set

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 20, 20, 64)] 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 20, 20, 64)   4160        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 20, 20, 64)   256         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 20, 20, 64)   0           batch_normalization[0][0]        
___________

# Transfer Learning results on Botswana

In [20]:
transfer_results

,Training samples per class finetuning,Training Samples per class pretraining,Training Samples,Test Samples,Test_Accuracies
0,15,200,210,2934,62.92
1,30,200,420,2724,62.92
2,45,200,630,2514,63.22
3,60,200,840,2304,63.22
4,75,200,1050,2094,63.60


# Classification accuracies per class for each model

In [21]:
confusion_matrixes[0]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,204,0,0,0,48,2,1,0,0,0,0,0,0,0,80.0
1,0,86,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,70,20,0,0,14,63,69,0,0,0,0,0,29.66
3,0,0,14,155,0,0,0,27,4,0,0,0,0,0,77.5
4,7,16,0,54,89,51,4,10,6,17,0,0,0,0,35.04
5,0,7,1,11,0,45,0,0,90,56,0,12,32,0,17.72
6,34,0,0,0,0,0,210,0,0,0,0,0,0,0,86.07
7,0,0,1,0,0,0,2,46,0,45,5,89,0,0,24.47
8,0,0,0,0,0,0,0,0,268,31,0,0,0,0,89.63
9,0,0,0,0,0,0,0,139,0,94,0,0,0,0,40.34


In [22]:
confusion_matrixes[1]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,190,0,0,0,50,0,0,0,0,0,0,0,0,0,79.17
1,0,71,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,39,29,0,0,14,80,59,0,0,0,0,0,17.65
3,0,0,0,176,0,0,0,9,0,0,0,0,0,0,95.14
4,8,12,0,70,109,13,3,10,0,14,0,0,0,0,45.61
5,0,0,0,35,0,44,0,0,70,32,0,8,50,0,18.41
6,34,0,0,0,0,0,195,0,0,0,0,0,0,0,85.15
7,0,0,0,5,0,0,0,28,0,24,38,78,0,0,16.18
8,0,0,0,0,0,0,0,0,268,16,0,0,0,0,94.37
9,0,0,0,0,0,0,0,150,0,68,0,0,0,0,31.19


In [23]:
confusion_matrixes[2]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,187,0,0,0,34,0,4,0,0,0,0,0,0,0,83.11
1,0,56,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,45,15,0,0,13,60,73,0,0,0,0,0,21.84
3,0,0,0,147,0,0,0,14,6,3,0,0,0,0,86.47
4,10,11,0,44,117,8,3,1,9,19,2,0,0,0,52.23
5,0,3,0,0,0,36,0,0,92,51,0,5,37,0,16.07
6,20,0,0,0,0,0,194,0,0,0,0,0,0,0,90.65
7,0,0,0,0,0,0,0,15,0,25,83,35,0,0,9.49
8,0,0,0,0,0,0,0,1,258,10,0,0,0,0,95.91
9,0,0,0,0,0,0,0,138,0,65,0,0,0,0,32.02


In [24]:
confusion_matrixes[3]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,172,0,0,0,34,0,4,0,0,0,0,0,0,0,81.9
1,0,41,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,55,5,0,0,11,37,83,0,0,0,0,0,28.8
3,0,0,0,76,0,0,0,23,47,9,0,0,0,0,49.03
4,9,6,0,22,143,3,4,0,12,8,2,0,0,0,68.42
5,0,3,0,0,0,24,0,0,97,38,0,5,42,0,11.48
6,19,0,0,0,0,0,180,0,0,0,0,0,0,0,90.45
7,0,0,0,0,5,0,0,0,0,20,95,23,0,0,0.0
8,0,0,0,0,0,0,0,0,236,18,0,0,0,0,92.91
9,0,0,0,0,0,0,0,96,0,92,0,0,0,0,48.94


In [25]:
confusion_matrixes[4]

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,classfication_accuracies
0,160,0,3,0,32,0,0,0,0,0,0,0,0,0,82.05
1,0,26,0,0,0,0,0,0,0,0,0,0,0,0,100.0
2,0,0,56,25,0,0,0,32,63,0,0,0,0,0,31.82
3,0,0,0,86,0,0,0,48,6,0,0,0,0,0,61.43
4,13,6,0,18,138,0,0,14,5,0,0,0,0,0,71.13
5,0,6,0,0,46,11,0,2,61,17,0,5,46,0,5.67
6,22,0,0,0,0,0,162,0,0,0,0,0,0,0,88.04
7,0,0,0,0,0,0,0,107,0,0,12,9,0,0,83.59
8,0,0,0,0,0,0,0,0,226,13,0,0,0,0,94.56
9,0,0,0,0,0,0,0,0,0,152,21,0,0,0,87.86
